# Ranzcr Clip - Catheter and Line Position Challenge

##  Fastai + Bayesian Optimization (Albumentation)

In short: It's all about identifing malpositioned lines and tubes in patients. More information about the challenge can be found [here](https://www.kaggle.com/c/ranzcr-clip-catheter-line-classification/overview)

In this notebook we will use Bayesian Optimization (with [hyperopt](https://github.com/hyperopt/hyperopt)) to optimize [Albumentations'](https://github.com/albumentations-team/albumentations) parameters in a [fastai](https://docs.fast.ai/) environment. 

Note that this notebook is just a small working example and serves only as a guildeline for the use of Bayesian Optimization with hyperopt. It should give you a feeling of how to use hyperopt + fastai + albumentation. Parameter values and selected Albumentation methods are chosen at random.

### Imports

In [ ]:
from fastai.vision.all import *
from fastai.metrics import accuracy_multi, RocAucMulti
import cv2
import pandas as pd
from sklearn.metrics import roc_auc_score
import albumentations
import warnings
warnings.simplefilter("ignore", FutureWarning)
from hyperopt import fmin, hp, tpe, Trials, STATUS_OK
import gc

In [ ]:
%cd ../input/efficientnetpytorch/EfficientNet-PyTorch-master
from efficientnet_pytorch import EfficientNet
%cd -

### Paths

In [ ]:
data_path = "../input/ranzcr-clip-catheter-line-classification/"
train_folder = data_path + "train/"

### Variables

In [ ]:
COL_NAMES = ['UID', 'ETTA','ETTB','ETTN','NGTA','NGTB','NGTI','NGTN','CVCA','CVCB','CVCN','SGCP', 'PatientID']

# Albumentation
RRC_SIZE = 512
RRC_MIN_SCALE = 0.75
RRC_RATIO = (1., 1.)
BRIGHTNESS_LIMIT = (-0.15,0.15)

# Augmentation
AUG_TRANS_SIZE = 256
AUG_TRANS_WARP = 0
AUG_TRANS_FLIP = True
AUG_TRANS_ROTATE = 20
AUG_TRANS_ZOOM = 1.2
AUG_TRANS_LIGHTNING = 0

# Model
EFFICIENTNET_PARAMS = ["efficientnet-b4", 1792]

# DataLoader
BS = 32

# Callbacks
PATIENCE_EARLY_STOPPING = 5

# Training. TODO: I chose such a small number of EPOCHS and FREEZE_EPOCHS just to create a small example.
EPOCHS = 2
FREEZE_EPOCHS = 2

## 1. Utils

### 1.1 Custom Transform

<u>RandomResizedCrop</u>
Transform images to same size by 1. resizing and 2. random crop

<u>Coarse Dropout / Cutout</u><br>
In short: Randomly remove rectangles from a given image. Where coarse dropout is removing many small rectangles of similar size and cutout is removing 1 large rectangle of random size

<u>Random Brightness</u><br>
Randomly change brightness of the image.

Note that we use different Transforms for Training and Validation (Testing). We will only apply a RandomResizedCrop on the Validation/ Testing Data.

In [ ]:
class AlbumentationsTransform(RandTransform):
    """
    A transform handler for multiple Albumentations transforms distinguishing between training 
    """
    split_idx, order = None, 2
    def __init__(self, train_aug, valid_aug):
        store_attr()
        
    def before_call(self, b, split_idx):
        self.idx = split_idx
    
    def encodes(self, img: PILImage):
        if self.idx == 0:
            aug_img = self.train_aug(image=np.array(img))['image']
        else:
            aug_img = self.valid_aug(image=np.array(img))['image']
        return PILImage.create(aug_img)
    
def get_train_aug(brightness_prob, coarse_prob, cutout_prob): 
    return albumentations.Compose([
        albumentations.RandomResizedCrop( 
            RRC_SIZE, RRC_SIZE,            
            scale=(RRC_MIN_SCALE, 1.0),
            ratio=RRC_RATIO,
            p=1.0
        ),
        albumentations.RandomBrightness(
            limit=BRIGHTNESS_LIMIT,
            p=brightness_prob
        ),
        albumentations.CoarseDropout(p=coarse_prob),
        albumentations.Cutout(p=cutout_prob)
    ])

def get_valid_aug(): 
    return albumentations.Compose([
        albumentations.Resize(RRC_SIZE, RRC_SIZE, p=1.0)
    ])

Short note on *aug_transforms*. It's a utility function which applies a list of transforms such as rotation, flipping etc **only** on the Training images (e.g. notice how *dls.valid.show_batch(nrows=1, ncols=5)* does not rotate and flip the images)

In [ ]:
batch_tfms = [*aug_transforms(size=AUG_TRANS_SIZE, max_warp=AUG_TRANS_WARP, do_flip=AUG_TRANS_FLIP,
                              max_rotate=AUG_TRANS_ROTATE, max_zoom=AUG_TRANS_ZOOM,
                              max_lighting=AUG_TRANS_LIGHTNING),
              Normalize.from_stats(*imagenet_stats)]

### 1.2 Metric

Let's define our own metric.

For each label/class we want to calculate the area under the receiver operating curve (here, in short: AUC). Further, we want another metric, taking the mean of each target's AUC score.

Probs to RobertLangdonVinci. Thanks for sharing your [Notebook](https://www.kaggle.com/robertlangdonvinci/fastai-efficientnetb5-custom-metrics)

In [ ]:
def mean_auc(preds, targs, labels=range(len(COL_NAMES)-2)):
    return np.mean([roc_auc_score(targs[:,i], preds[:,i]) for i in labels])
def ETTA_auc(*args):
    return mean_auc(*args, labels=[0])
def ETTB_auc(*args):
    return mean_auc(*args, labels=[1])
def ETTN_auc(*args):
    return mean_auc(*args, labels=[2])
def NGTA_auc(*args):
    return mean_auc(*args, labels=[3])
def NGTB_auc(*args):
    return mean_auc(*args, labels=[4])
def NGTI_auc(*args):
    return mean_auc(*args, labels=[5])
def NGTN_auc(*args):
    return mean_auc(*args, labels=[6])
def CVCA_auc(*args):
    return mean_auc(*args, labels=[7])
def CVCB_auc(*args):
    return mean_auc(*args, labels=[8])
def CVCN_auc(*args):
    return mean_auc(*args, labels=[9])
def SGCP_auc(*args):
    return mean_auc(*args, labels=[10])

In [ ]:
metrics = [ AccumMetric(mean_auc, flatten=False),
            AccumMetric(ETTA_auc, flatten=False),
            AccumMetric(ETTB_auc, flatten=False),
            AccumMetric(ETTN_auc, flatten=False),
            AccumMetric(NGTA_auc, flatten=False),
            AccumMetric(NGTB_auc, flatten=False),
            AccumMetric(NGTI_auc, flatten=False),
            AccumMetric(NGTN_auc, flatten=False),
            AccumMetric(CVCA_auc, flatten=False),
            AccumMetric(CVCB_auc, flatten=False),
            AccumMetric(CVCN_auc, flatten=False),
            AccumMetric(SGCP_auc, flatten=False), 
            accuracy_multi]

### 1.3 Model

Create a Learner based on a pretrained EfficientNet-B4 model.

In [ ]:
class RanzerModel(Module):
    def __init__(self, num_classes):
        self.effnet = EfficientNet.from_pretrained(EFFICIENTNET_PARAMS[0], weights_path=None, include_top=False)
        self.dropout = nn.Dropout(0.1)
        self.out = nn.Linear(EFFICIENTNET_PARAMS[1], num_classes)

    def forward(self, image):
        batch_size, _, _, _ = image.shape
        x = self.effnet.extract_features(image)
        x = F.adaptive_avg_pool2d(x, 1).reshape(batch_size, -1)
        outputs = self.out(self.dropout(x))
        return outputs

### 1.4 Load Data

In [ ]:
# Read training set
df_train = pd.read_csv(data_path + "train.csv")
df_train.columns = COL_NAMES

# Add entire path such that DataLoader knows the path for each file
df_train['path'] = df_train['UID'].map(lambda x:str(train_folder + x)+'.jpg')
df_train = df_train.drop(columns=['UID'])

### 1.5 Callbacks

More information on Callbacks can be found [here](https://docs.fast.ai/callback.core.html)

In [ ]:
cb1 = SaveModelCallback(monitor='mean_auc', fname='best-model', comp=np.greater)
cb2 = EarlyStoppingCallback(monitor='valid_loss', min_delta=0.0, patience=PATIENCE_EARLY_STOPPING)

## 2. Training

### 2.1 Search Space

Define the search space. Provide parameters and ranges of values using hyperopts stochastic expressions such as <br></br>


* hp.choice: Returns one of the options
* hp.randint: Returns a random integer in the range [0, upper)
* hp.uniform: Returns a value uniformly between two variables

See https://github.com/hyperopt/hyperopt/wiki/FMin#21-parameter-expressions

In [ ]:
search_space = hp.choice('classifier',[
    {
        'param': {'brightness_prob': hp.uniform('brightness_prob', 0.0, 1.0),
                  'coarse_prob': hp.uniform('coarse_prob', 0.0, 1.0),
                  'cutout_prob': hp.uniform('cutout_prob', 0.0, 1.0)
                 }
    }
])

### 2.2 Define Objective Function

We need to define a function which should be **minimized** (that is why we are returning the negative mean AUC). Just place your model training inside such a function.

In [ ]:
def hyperparameter_tuning(params):
    
    """
    Objective function

    It takes in hyperparameter settings, fits a model based on those settings,
    evaluates the model, and returns the mean AUC score.

    :param params: map specifying the hyperparameter settings to test
    :return: mean AUC for the fitted model
    """
    
    print("Parameter: {}".format(params['param']))
    brightness_prob = params['param']['brightness_prob']
    coarse_prob = params['param']['coarse_prob']
    cutout_prob = params['param']['cutout_prob']
    
    item_tfms = AlbumentationsTransform(get_train_aug(brightness_prob, coarse_prob, cutout_prob),
                                        get_valid_aug())
    
    data = DataBlock(blocks=(ImageBlock, MultiCategoryBlock(encoded=True, vocab=list(df_train.columns[:11]))),
                 splitter = RandomSplitter(seed=123),
                 get_x = ColReader(12),
                 get_y = ColReader(list(range(11))),
                 item_tfms = item_tfms,
                 batch_tfms = batch_tfms,
                )

    dls = data.dataloaders(df_train, bs=BS)
    efficientnet = RanzerModel(11)
    learn = Learner(dls, efficientnet, metrics=metrics, opt_func=Adam, cbs=[cb1, cb2])
    learn.to_native_fp16()
    
    learn.fine_tune(epochs=EPOCHS, base_lr=2e-3, freeze_epochs=FREEZE_EPOCHS)
    
    auc_mean = float(learn.validate(dl=dls.valid)[2])
    
    del learn
    torch.cuda.empty_cache()
    gc.collect()
  

    return {'loss': -auc_mean, 'status': STATUS_OK}

### 2.3 Optimization Loop

Save statistics with the help of a Trials object. More information can be found [here](https://github.com/hyperopt/hyperopt/wiki/FMin#12-attaching-extra-information-via-the-trials-object).

In short: We are using the Trials object to store information such as the AUC Mean score of each evaluation and whether the evaluation went well.

In [ ]:
trials = Trials()

argmin = fmin(fn=hyperparameter_tuning,  # Objective function
              space=search_space,  # Search space
              algo=tpe.suggest,  # Use the tree of Parzen estimators:(Bayesian optimization). Alternative: random.suggest (Random Search)
              trials=trials, # Trials object
              max_evals=2 # Maximum number of evaluations/trials (hyperparameter settings)
             )
print(argmin)

## 3. Interpretation

*argmin* contains the parameters for the model which lead to the highest auc mean.

In this case it is:
* a brightness probability of 0.38
* a coarse probability of 0.91
* a cutout probability of 0.86

In [ ]:
print(argmin)